In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import re

In [2]:
import urllib3 

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
login_url='https://www.jobplanet.co.kr/users/sign_in'

email = 'list456@ajou.ac.kr'
password = "70909skdus!"

LOGIN_INFO = {
    'user[email]' : email,
    'user[password]' : password,
    'commit' : '로그인'
}

session = requests.session()

res = session.post(login_url, data = LOGIN_INFO, verify = False)

res.raise_for_status()

In [4]:
# 총 기업 개수 반환하는 함수
def company_count():
  url = 'https://www.jobplanet.co.kr/companies?industry_id=900'
  r = session.get(url)
  r.raise_for_status()

  soup = BeautifulSoup(r.text, 'html.parser')
  #print(soup)
  last_num_html = soup.find('span', {'class': 'num'})
  last_num = last_num_html.get_text()
  return int(last_num)

In [5]:
# 페이지 수 반환하는 함수
def last_page(x):
  if x % 10 == 0:
    last_page_num = x // 10
  else :
    last_page_num = (x // 10) + 1

  return last_page_num

In [6]:
# 마지막 페이지
company_num = company_count()
last_page = last_page(company_num)
print(last_page)

65


In [7]:
number = []
company_name = []
links = []
# for k in range(1, last_page+1):  현대카드만 찾은 거라 바꾸기

for k in range(1, 2):
  url = 'https://www.jobplanet.co.kr/companies?&industry_id=900&page=' + str(k)
  r1 = session.get(url)
  r1.raise_for_status()


  soup1 = BeautifulSoup(r1.text, 'html.parser')

  for i in soup1.find_all('dt', attrs={'class': 'us_titb_l3'}):
    for j in i.find_all('a'):
      number_match = re.search(r'/companies/(\d+)/', j['href']) # search 구문 공부 숫자 뽑아내는 코드
      if number_match:
          number.append(number_match.group(1))
      company_name.append(j.text.strip())

      review_url = 'https://www.jobplanet.co.kr/companies/' + number_match.group(1) + '/interviews/' + j.text.strip()
      links.append(review_url)

print(number)
print(company_name)
print(links)

['44440', '89935', '43917', '41466', '1227', '55770', '44434', '1225', '85166', '30195']
['국민건강보험공단', '새마을금고', '근로복지공단', '농협은행(주)', '현대카드(주)', '중소기업은행', '국민연금공단', '현대캐피탈(주)', '국민은행(주)', '삼성생명보험(주)']
['https://www.jobplanet.co.kr/companies/44440/interviews/국민건강보험공단', 'https://www.jobplanet.co.kr/companies/89935/interviews/새마을금고', 'https://www.jobplanet.co.kr/companies/43917/interviews/근로복지공단', 'https://www.jobplanet.co.kr/companies/41466/interviews/농협은행(주)', 'https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)', 'https://www.jobplanet.co.kr/companies/55770/interviews/중소기업은행', 'https://www.jobplanet.co.kr/companies/44434/interviews/국민연금공단', 'https://www.jobplanet.co.kr/companies/1225/interviews/현대캐피탈(주)', 'https://www.jobplanet.co.kr/companies/85166/interviews/국민은행(주)', 'https://www.jobplanet.co.kr/companies/30195/interviews/삼성생명보험(주)']


In [8]:
links[4]

'https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)'

In [9]:
# 회사이름, 링크, 넘버 리스트 --> 데이터 프레임으로 변환하고 csv 파일로 저장하기
df_company_name = pd.DataFrame(company_name)
df_number = pd.DataFrame(number)
df_links = pd.DataFrame(links)

In [10]:
#df_company_name.to_csv('company_name.csv')
df_number.to_csv('number_f.csv', index = False, encoding = 'utf-8-sig')
df_links.to_csv('links_f.csv', index = False, encoding = 'utf-8-sig')
df_company_name.to_csv('company_name_f.csv', index=False, encoding = 'utf-8-sig')

In [11]:
# 상세페이지 - 페이지 수 확인하는 함수 (기업에 해당하는 url 넘겨주기)
def reviews_count(url):
  r = session.get(url)
  r.raise_for_status()

  soup = BeautifulSoup(r.text, 'html.parser')
  review = soup.find('div', {'id': 'viewInterviewsTitle'}).find('span', {'class': 'num'})

  num = review.get_text()
  num = int(num.replace(',', ''))
  return num

In [12]:
# 페이지 계산하는 함수
def reviews_page_count(x):
  if x % 5 == 0:
    reviews_page_num = x // 5
  else:
    reviews_page_num = (x // 5) + 1

  return reviews_page_num

In [13]:
# 면접시기, 느낌 빈 값 처리 함수
def insert_to_list(original_list, index):
    
    tmp_list = []
    
    while len(original_list) > index:
        tmp_list.append(original_list.pop())

    original_list.extend(['N'])
    
    original_list.extend(tmp_list[::-1])
    
    return original_list

In [14]:
len(links)

10

In [15]:
Date_list = []
career_list = []
edu_list = []
question_list = []
feel_list = []
Dates_list = []
Feel_list = []

# 리뷰 처음 체이지
for href in links[4:5]:
  url = href # 특정 회사 페이지 도착
  review_num = reviews_count(url)
  review_page_num = reviews_page_count(review_num)

  for i in range(1, review_page_num+1):
    urld = url + '?page=' + str(i)
    print(urld)

    rew = session.get(urld, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'})
    rew.raise_for_status()

    soup1 = BeautifulSoup(rew.text, 'html.parser')

    company_name = re.search(r'interviews/([^/]+)',url).group(1)


    for i in soup1.find_all("dl", class_="ctbody_lft"):
      for kk in i.find_all("dd", class_="txt1"):
        date = kk.get_text().strip()

        Date_list.append([company_name, date])

    for i in soup1.find_all("dl", class_="ctbody_lft"):
      k = i.find_all("dd", class_="txt1")

      Dates_list.append([company_name, k])
    
    for k in soup1.find_all('span', {'class' : 'txt1'}):
      careers = k.get_text().strip().split('\n')[0]
      career_list.append([company_name, careers])

    for p in soup1.find_all('span', {'class' : 'txt1'}):
      edu1 = p.get_text().strip().split('\n')[2].strip()
      edu_list.append([company_name, edu1])

    for p in soup1.find_all('span', {'class': 'answer mobile_full_content notranslate'}):
      feel = p.get_text()
      
      feel_list.append((company_name, feel))

    for i in soup1.find_all("dl", class_="tc_list"):
      col = i.find_all('span', {'class': 'answer mobile_full_content notranslate'})

      Feel_list.append([company_name, col])

  
    for i in soup1.find_all("dl", class_="tc_list"):
      o = i.find("dd", class_="df1").get_text().strip().split('\n')

      question_list.append([company_name, o])



https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=1
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=2
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=3
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=4
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=5
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=6
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=7
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=8
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=9
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=10
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=11
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=12
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=13
https://www.jobplanet.co.kr/companies/1227/interviews/현대카드(주)?page=14
https://www.jobplanet.co.kr/c

In [16]:
# 면접 일자와 느낌 빈 값 확인하도록 비교 리스트 생성, 빈 값 인덱스 추출
for i in range(len(Dates_list)):
   if Dates_list[i][1] == []:
      Dates_list[i][1] = ['noday']

b = [p[1] for p in Dates_list]

rest_list = list(filter(lambda x: b[x] == ['noday'], range(len(b))))


for i in range(len(Feel_list)):
   if Feel_list[i][1] == []:
      Feel_list[i][1] = ['no']

c = [q[1] for q in Feel_list]

Rest_list = list(filter(lambda x: c[x] == ['no'], range(len(c))))

In [17]:
# 빈 값 인덱스에 맞춰 넣기
for i in rest_list:
    insert_to_list(Date_list, i)

for j in Rest_list:
    insert_to_list(feel_list, j)

In [18]:
Date_df = pd.DataFrame(Date_list, columns=['company_name','date'])
career_df = pd.DataFrame(career_list, columns=['company_name', 'career'])
edu_df = pd.DataFrame(edu_list, columns=['company_name', 'edu'])
question_df = pd.DataFrame(question_list, columns=['company_name', 'question'])
feel_df = pd.DataFrame(feel_list, columns=['company_name', 'feel'])
Dates_df = pd.DataFrame(Dates_list, columns=['company_name','DATE'])
Feel_df = pd.DataFrame(Feel_list, columns=['company_name', 'Feel'])


df = pd.concat([career_df, Date_df['date'], edu_df['edu'], question_df['question'], feel_df['feel']], axis=1)
df['feel'] = df['feel'].str.replace('\n', '').str.strip().str.replace('\r', ' ').str.replace('-', ' ').str.replace('=', '').str.strip()
df['question'] = df['question'].str[0].str.strip('[]').str.replace('\r', ' ').str.replace('-', ' ').str.replace('=', '').str.strip()

df

,company_name,career,date,edu,question,feel
0,현대카드(주),금융/재무,2023/07,기타,지원동기를 말하세요. 이전직장에서 했던 일은 무엇인가요? 자기소개서 위주로 질문하였습니다,딱딱한 분위기 보다는 부드러운 분위기로 면접을 진행하려고 애써주셨습니다.
1,현대카드(주),생산관리/품질관리,2018/02,사원-고졸,1. 면접에 양복을 입고 오지 않은 이유 2. 군대는다녀 왔냐 3. 스트레스 받을때...,솔직히 계약직 단순 업무 자리였기 때문에 어려운 면접은 아니였습니다. 아르바이트 면...
2,현대카드(주),영업/제휴,2022/04,사원-고졸,1. 자기소개 2. 이전에 다녔던 회사에서 재직기간이 짧은 이유 3. 그곳에서 어...,코로나 심했을 때 화상면접으로 진행했던 거라 부담이 덜했고 분위기는 좋았습니다
3,현대카드(주),영업/제휴,2022/10,사원-대졸,제일 힘들었던 기억을 어떻게 이겨내었는지? 살면서 가장 큰 성과는? 영업이란 무엇인가?,"학창시절 힘들었던 것을 혼자스스로 이거낸 경험, 아무것도 없이 시작하여 매출을 냈던..."
4,현대카드(주),기획/경영,2020/06,사원-대졸,현대카드가 하면 좋을것 같은 사업은 무엇이 있을지 아이디에이션 요청,당시 당근마켓 카드결제가 없어서 당근마켓과의 plcc 혹은 카드결제 서비스 탑재 이야기함
...,...,...,...,...,...,...
646,현대카드(주),기획/경영,None,사원-대졸,자료의 이해,None
647,현대카드(주),개발,2014/04,과장,돈과 권력이 모두 주어진다면 가장먼저 하고싶은일은?,현대카드 포인트몰 리뉴얼
648,현대카드(주),영업/제휴,2014/03,사원-대졸,제조업 영업관리와 금융업 영업관리가 어떻게 다른가?,None
649,현대카드(주),기획/경영,2013/02,사원-대졸,미래에 관한 질문,None


In [19]:
# 인덱스 포함 csv 파일
df.to_csv('interviews_index.csv', encoding='utf-8-sig', escapechar='/')

In [20]:
# 인덱스 미포함 csv 파일
df.to_csv('interviews_no_index.csv', index=False, encoding='utf-8-sig', escapechar='/')

In [21]:
hc = pd.read_csv(r'C:\Users\tarbo\src\project\interviews_index.csv')
df2 = pd.DataFrame(hc)
df2

,Unnamed: 0,company_name,career,date,edu,question,feel
0,0,현대카드(주),금융//재무,2023//07,기타,지원동기를 말하세요. 이전직장에서 했던 일은 무엇인가요? 자기소개서 위주로 질문하였습니다,딱딱한 분위기 보다는 부드러운 분위기로 면접을 진행하려고 애써주셨습니다.
1,1,현대카드(주),생산관리//품질관리,2018//02,사원-고졸,1. 면접에 양복을 입고 오지 않은 이유 2. 군대는다녀 왔냐 3. 스트레스 받을때...,솔직히 계약직 단순 업무 자리였기 때문에 어려운 면접은 아니였습니다. 아르바이트 면...
2,2,현대카드(주),영업//제휴,2022//04,사원-고졸,1. 자기소개 2. 이전에 다녔던 회사에서 재직기간이 짧은 이유 3. 그곳에서 어...,코로나 심했을 때 화상면접으로 진행했던 거라 부담이 덜했고 분위기는 좋았습니다
3,3,현대카드(주),영업//제휴,2022//10,사원-대졸,제일 힘들었던 기억을 어떻게 이겨내었는지? 살면서 가장 큰 성과는? 영업이란 무엇인가?,"학창시절 힘들었던 것을 혼자스스로 이거낸 경험, 아무것도 없이 시작하여 매출을 냈던..."
4,4,현대카드(주),기획//경영,2020//06,사원-대졸,현대카드가 하면 좋을것 같은 사업은 무엇이 있을지 아이디에이션 요청,당시 당근마켓 카드결제가 없어서 당근마켓과의 plcc 혹은 카드결제 서비스 탑재 이야기함
...,...,...,...,...,...,...,...
646,646,현대카드(주),기획//경영,NaN,사원-대졸,자료의 이해,NaN
647,647,현대카드(주),개발,2014//04,과장,돈과 권력이 모두 주어진다면 가장먼저 하고싶은일은?,현대카드 포인트몰 리뉴얼
648,648,현대카드(주),영업//제휴,2014//03,사원-대졸,제조업 영업관리와 금융업 영업관리가 어떻게 다른가?,NaN
649,649,현대카드(주),기획//경영,2013//02,사원-대졸,미래에 관한 질문,NaN
